In [1]:
import pandas as pd

In [37]:
# get the raw data 
df = pd.read_csv('games_details.csv')
games = pd.read_csv('games.csv')
all_stars_raw = pd.read_csv('all_stars_raw.csv', header=None)
players_df = pd.read_csv('players.csv')

/var/folders/m0/l6jpq4w579906wrx6f1v04700000gn/T/ipykernel_47349/446194918.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('games_details.csv')


In [27]:
# dedup the all stars and get their ids
all_stars = all_stars_raw.drop_duplicates()
all_stars = all_stars.drop([2,3], axis=1)
all_stars.columns = ['PLAYER_NAME', 'hi']
# get the player id from the players_df
all_stars = pd.merge(all_stars, players_df, on='PLAYER_NAME', how='left')
all_stars = all_stars.drop(['hi', 'TEAM_ID', 'SEASON'], axis=1)
all_stars = all_stars.drop_duplicates()
all_stars = all_stars.dropna()

all_stars.to_csv('all_stars.csv', index=False)

In [42]:
from tqdm import tqdm
# split the data into two parts: playoffs and not playoffs
# add the date of the game to the df
df = pd.read_csv('games_details.csv')
df = pd.merge(df, games[['GAME_ID', 'GAME_DATE_EST']], on='GAME_ID', how='left')

playoff_starts = ['2012-04-28', '2013-04-20', '2014-04-19', '2015-04-18', '2016-04-16', '2017-04-15', '2018-04-14', '2019-04-13', '2020-08-17', '2021-05-22', '2022-04-16']
playoff_ends = ['2012-06-21', '2013-05-05', '2014-06-15', '2015-06-16', '2016-06-19', '2017-06-12', '2018-06-08', '2019-06-13', '2020-10-11', '2021-07-20', '2022-06-16']

#create 2 empty dataframes with the same columns as df
df_playoffs = pd.DataFrame(columns=df.columns)
df_regular = pd.DataFrame(columns=df.columns)

df_playoffs_list = []
df_regular_list = []

# include the game if it is between the start and end date of any index in the playoff_starts list
for i, row in tqdm(df.iterrows()):
    # if i % 1000 == 0:
    #     print(i)
    for j in range(len(playoff_starts)):
        if row['GAME_DATE_EST'] >= playoff_starts[j] and row['GAME_DATE_EST'] <= playoff_ends[j]:
            df_playoffs_list.append(row)
            break
        if j == (len(playoff_starts) - 1):
            df_regular_list.append(row)

df_playoffs = pd.DataFrame(df_playoffs_list)    
df_regular = pd.DataFrame(df_regular_list)

# save these two dataframes
df_playoffs.to_csv('games_details_playoffs.csv', index=False)
df_regular.to_csv('games_details_regular.csv', index=False)


/var/folders/m0/l6jpq4w579906wrx6f1v04700000gn/T/ipykernel_47349/3516768417.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('games_details.csv')
669560it [00:23, 28605.80it/s]


In [62]:
# get the regular season and playoffs data
df_regular = pd.read_csv('games_details_regular.csv')
df_playoffs = pd.read_csv('games_details_playoffs.csv')

# drop rows that have 'MIN' column as NaN
df_regular = df_regular.dropna(subset=['MIN'])
df_playoffs = df_playoffs.dropna(subset=['MIN'])

# we need to convert the minutes played to a float
def convert_minutes_to_float(minutes):
    if minutes == 0:
        return 0
    else:
        # round to the nearest minute
        minutes = minutes.split(':')
        if len(minutes) == 1:
            return int(float(minutes[0]))
        return int(float(minutes[0])) + (int(float(minutes[1])) > 30)
    
df_regular['MIN'] = df_regular['MIN'].apply(convert_minutes_to_float)
df_playoffs['MIN'] = df_playoffs['MIN'].apply(convert_minutes_to_float)

df_playoffs.to_csv('games_details_playoffs.csv', index=False)
df_regular.to_csv('games_details_regular.csv', index=False)

/var/folders/m0/l6jpq4w579906wrx6f1v04700000gn/T/ipykernel_47349/4165856932.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regular = pd.read_csv('games_details_regular.csv')


In [71]:
# now for each all star, we want to get the average minutes played, points, rebounds, assists, steals, blocks, turnovers, and plus minus for the regular season and playoffs respectively

# get the all stars
all_stars = pd.read_csv('all_stars.csv')

# get the regular season and playoffs data
df_regular = pd.read_csv('games_details_regular.csv')
df_playoffs = pd.read_csv('games_details_playoffs.csv')

# we need to convert the minutes played to a float

# create a dataframe to store the averages
all_stars_averages = pd.DataFrame(columns=['PLAYER_NAME', 'PLAYER_ID', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
                                           'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'PTS', 'OREB', 'DREB', 'REB', 'AST', 'STL', 
                                           'BLK', 'TO', 'PF', 'PLUS_MINUS', 'SEASON_TYPE', 'GAMES_PLAYED'])

# for each player id, get the average stats for the regular season and playoffs
for i, row in tqdm(all_stars.iterrows()):
    player_id = row['PLAYER_ID']

    # get the regular season stats
    player_regular = df_regular[df_regular['PLAYER_ID'] == player_id]
    # get the playoffs stats
    player_playoffs = df_playoffs[df_playoffs['PLAYER_ID'] == player_id]

    # get the averages
    player_regular_avg = player_regular[all_stars_averages.columns[2: -2]].mean()
    player_playoffs_avg = player_playoffs[all_stars_averages.columns[2: -2]].mean()

    # add the player id
    player_regular_avg['PLAYER_ID'] = player_id
    player_playoffs_avg['PLAYER_ID'] = player_id

    # add the player name
    player_regular_avg['PLAYER_NAME'] = row['PLAYER_NAME']
    player_playoffs_avg['PLAYER_NAME'] = row['PLAYER_NAME']

    # add the season type
    player_regular_avg['SEASON_TYPE'] = 'Regular'
    player_playoffs_avg['SEASON_TYPE'] = 'Playoffs'

    # we also need to include the number of games played
    player_regular_avg['GAMES_PLAYED'] = len(player_regular)
    player_playoffs_avg['GAMES_PLAYED'] = len(player_playoffs)

    # if the player didn't play at least 30 playoff games we don't want to include them
    if len(player_playoffs) < 30:
        continue

    # add these to the dataframe
    all_stars_averages.loc[len(all_stars_averages)] = player_regular_avg
    all_stars_averages.loc[len(all_stars_averages)] = player_playoffs_avg

# save the dataframe
all_stars_averages.to_csv('all_stars_averages.csv', index=False)

/var/folders/m0/l6jpq4w579906wrx6f1v04700000gn/T/ipykernel_47349/341626614.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regular = pd.read_csv('games_details_regular.csv')
77it [00:00, 215.94it/s]
